In [ ]:
%matplotlib notebook
import json
import logging
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import ipywidgets as widgets
from pprint import pprint
from nupic.research.frameworks.pytorch.imagenet_experiment import ImagenetExperiment
from projects.imagenet.experiments_superconvergence import CONFIGS
logging.disable(sys.maxsize)

# Select experiment 

In [ ]:
try:
    name = selection.value
except:
    name = "default"
selection = widgets.Dropdown(options=CONFIGS.keys(), value=name)
selection

In [ ]:
name = selection.value
config = CONFIGS[name]
config.update(name=name)
pprint(config)

# Compute all LR values by simulating training

In [ ]:
def compute_lr(exp):
    lr = []
    total_images = len(exp.train_loader.dataset)
    for epoch in range(exp.epochs):
        exp.pre_epoch(epoch)
        total_batches = -(-total_images // exp.batch_size)
        for batch in range(total_batches):
            exp.pre_batch(exp.model, batch, epoch)
            exp.optimizer.step()
            lr.append(exp.get_lr()[0])
            exp.post_batch(exp.model, 0, batch, epoch)
        exp.post_epoch(epoch)
    return lr

exp = ImagenetExperiment()
exp.setup_experiment(config)
lr = compute_lr(exp)

# Plot experiment

In [ ]:
plt.plot(lr, label=name)
plt.title("Leaning rate schedule")
plt.xlabel("batches")
plt.ylabel("learning rate")
plt.legend()
plt.show()

# Save results

In [ ]:
# Save config
with open(f"{name}.json", "w") as fp:
    json.dump(config, fp, default=lambda o: str(o))

# Save LR by epochs
lr_per_epoch = np.array_split(lr, exp.epochs)
df = pd.DataFrame(lr_per_epoch, index=range(len(lr_per_epoch)), columns=range(len(lr_per_epoch[0])))
df.to_csv(f"{name}.csv", header=False)

# Show LR by epochs table (slow)

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
df